In [1]:
import time,os,json
import numpy as np
import matplotlib.pyplot as plt

from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cs231n.rnn_layers import *
from cs231n.captioning_solver import CaptioningSolver
from cs231n.classifiers.rnn import CaptioningRNN
from cs231n.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from cs231n.image_utils import image_from_url

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0,8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

def rel_error(x,y):
    return np.max(np.abs(x-y)/(np.maximum(1e-8,np.abs(x)+np.abs(y))))

run the following from the cs231n directory and try again:
python setup.py build_ext --inplace
You may also need to restart your iPython kernel


In [2]:
data = load_coco_data(pca_features=True)

for k,v in data.items():
    if type(v) == np.ndarray:
        print(k,type(v),v.shape,v.dtype)
    else:
        print(k,type(v),len(v))

train_urls <class 'numpy.ndarray'> (82783,) <U63
val_urls <class 'numpy.ndarray'> (40504,) <U63
word_to_idx <class 'dict'> 1004
train_features <class 'numpy.ndarray'> (82783, 512) float32
train_captions <class 'numpy.ndarray'> (400135, 17) int32
idx_to_word <class 'list'> 1004
val_features <class 'numpy.ndarray'> (40504, 512) float32
val_image_idxs <class 'numpy.ndarray'> (195954,) int32
val_captions <class 'numpy.ndarray'> (195954, 17) int32
train_image_idxs <class 'numpy.ndarray'> (400135,) int32


In [3]:
batch_size = 3

captions,features,urls = sample_coco_minibatch(data,batch_size=batch_size)
for i,(caption, url) in enumerate(zip(captions,urls)):
    plt.imshow(image_from_url(url))
    plt.axis('off')
    caption_str = decode_captions(caption, data['idx_to_word'])
    plt.title(caption_str)
    plt.show()

ConnectionResetError: [Errno 54] Connection reset by peer

In [ ]:
N, D, H = 3, 10, 4

x = np.linspace(-0.4,0.7,num=N*D).reshape(N,D)
prev_h = np.linspace(-0.2,0.5,num=N*H).reshape(N,H)
Wx = np.linspace(-0.1,0.9,num=D*H).reshape(D,H)
Wh = np.linspace(-0.3,0.7,num=H*H).reshape(H,H)
b = np.linspace(-0.2,0.4,num=H)

next_h,_ = rnn_step_forward(x,prev_h,Wx,Wh,b)
expected_next_h = np.asarray([
    [-0.58172089, -0.50182032, -0.41232771, -0.31410098],
    [0.66854692, 0.79562378, 0.87755553, 0.92795967],
    [0.97934501, 0.99144213, 0.99646691, 0.99854353]])

print('next_h error: ',rel_error(expected_next_h,next_h))

In [ ]:
from cs231n.rnn_layers import rnn_step_forward, rnn_step_backward

N,D,H = 4,5,6
x = np.random.randn(N,D)
h = np.random.randn(N,H)
Wx = np.random.randn(D,H)
Wh = np.random.randn(H,H)
b = np.random.randn(H)

out, cache = rnn_step_forward(x,h,Wx,Wh,b)

dnext_h = np.random.randn(*out.shape)

fx = lambda x: rnn_step_forward(x,h,Wx,Wh,b)[0]
fh = lambda h: rnn_step_forward(x,h,Wx,Wh,b)[0]
fWx = lambda Wx: rnn_step_forward(x,h,Wx,Wh,b)[0]
fWh = lambda Wh: rnn_step_forward(x,h,Wx,Wh,b)[0]
fb = lambda b: rnn_step_forward(x,h,Wx,Wh,b)[0]

dx_num = eval_numerical_gradient_array(fx,x,dnext_h)
dprev_h_num = eval_numerical_gradient_array(fh,h,dnext_h)
dWx_num = eval_numerical_gradient_array(fWx,Wx,dnext_h)
dWh_num = eval_numerical_gradient_array(fWh,Wh,dnext_h)
db_num = eval_numerical_gradient_array(fb,b,dnext_h)

dx,dprev_h,dWx,dWh,db = rnn_step_backward(dnext_h,cache)

print('dx error: ',rel_error(dx_num,dx))
print('dprev error: ', rel_error(dprev_h_num,dprev_h))
print('dWx error: ',rel_error(dWx_num,dWx))
print('dWh error: ',rel_error(dWh_num,dWh))
print('db error: ',rel_error(db_num,db))

In [ ]:
N,T,V,D = 2,4,5,3

x = np.asarray([[0,3,1,2],[2,1,0,3]])
W = np.linspace(0,1,num=V*D).reshape(V,D)

out, _= word_embedding_forward(x,W)
expected_out = np.asarray([
    [[0.        ,0.07142857,0.14285714],
     [0.64285714,0.71428571,0.78571429],
     [0.21428571,0.28571429,0.35714286],
     [0.42857143,0.5,       0.57142857]],
    [[0.42857143,0.5,       0.57142857],
     [0.21428571,0.28571429,0.35714286],
     [0.,        0.07142857,0.14285714],
     [0.64285714,0.71428571,0.78571429]]])
print('out error: ',rel_error(expected_out,out))

In [ ]:
N,T,V,D = 50,3,5,6
x = np.random.randint(V,size=(N,T))
W = np.random.randn(V,D)
out, cache = word_embedding_forward(x,W)
dout = np.random.randn(*out.shape)
dW = word_embedding_backward(dout, cache)

f = lambda W:word_embedding_forward(x,W)[0]
dW_num = eval_numerical_gradient_array(f,W,dout)
print('dW error: ',rel_error(dW,dW_num))

In [ ]:
N,T,D,M = 2,3,4,5

x = np.random.randn(N,T,D)
w = np.random.randn(D,M)
b = np.random.randn(M)

out,cache = temporal_affine_forward(x,w,b)

dout = np.random.randn(*out.shape)

fx = lambda x: temporal_affine_forward(x,w,b)[0]
fw = lambda w: temporal_affine_forward(x,w,b)[0]
fb = lambda b: temporal_affine_forward(x,w,b)[0]

dx_num = eval_numerical_gradient_array(fx,x,dout)
dw_num = eval_numerical_gradient_array(fw,w,dout)
db_num = eval_numerical_gradient_array(fb,b,dout)

dx, dw, db = temporal_affine_backward(dout,cache)

print('dx error: ',rel_error(dx_num,dx))
print('dw error: ',rel_error(dw_num,dw))
print('db error: ',rel_error(db_num,db))

In [ ]:
from cs231n.rnn_layers import temporal_softmax_loss

N,T,V = 100,1,10

def check_loss(N,T,V,p):
    x = 0.001 * np.random.randn(N,T,V)
    y = np.random.randint(V,size=(N,T))
    mask = np.random.rand(N,T) <= p
    print(temporal_softmax_loss(x,y,mask)[0])
    
check_loss(100,1,10,1.0)
check_loss(100,10,10,1.0)
check_loss(5000,10,10,0.1)

N,T,V = 7,8,9

x = np.random.randn(N,T,V)
y = np.random.randint(V,size=(N,T))
mask = (np.random.rand(N,T) > 0.5)

loss, dx = temporal_softmax_loss(x,y,mask,verbose=False)

dx_num = eval_numerical_gradient(lambda x: temporal_softmax_loss(x,y,mask)[0],x,verbose=False)

print('dx error: ',rel_error(dx,dx_num))

In [ ]:
N,D,W,H = 10,20,30,40
word_to_idx = {'<NULL>':0,'cat':2,'dog':3}
V = len(word_to_idx)
T = 13

model = CaptioningRNN(word_to_idx,
                     input_dim=D,
                     wordvec_dim=W,
                     hidden_dim=H,
                     cell_type='rnn',
                     dtype=np.float64)

for k,v in model.params.items():
    model.params[k] = np.linspace(-1.4,1.3,num=v.size).reshape(*v.shape)
    
features = np.linspace(-1.5,0.3,num=(N*D)).reshape(N,D)
captions = (np.arange(N*T)%V).reshape(N,T)

loss,grads = model.loss(features,captions)
expected_loss = 9.83235591003

print('loss: ',loss)
print('expected loss: ',expected_loss)
print('difference: ',abs(loss-expected_loss))

In [ ]:
batch_size = 2
timesteps = 3
input_dim = 4
wordvec_dim = 5
hidden_dim = 6
word_to_idx = {'<NULL>':0,'cat':2,'dog':3}
vocab_size = len(word_to_idx)

captions = np.random.randint(vocab_size,size=(batch_size,timesteps))
features = np.random.randn(batch_size,input_dim)

model = CaptioningRNN(word_to_idx,
                     input_dim=input_dim,
                     wordvec_dim=wordvec_dim,
                     hidden_dim=hidden_dim,
                     cell_type='rnn',
                     dtype=np.float64,
                     )

loss, grads = model.loss(features,captions)

for param_name in sorted(grads):
    f = lambda _:model.loss(features,captions)[0]
    param_grad_num = eval_numerical_gradient(f,model.params[param_name],verbose=False,h=1e-5)
    e = rel_error(param_grad_num,grads[param_name])
    print('%s relative error: %e' %(param_name,e))

In [ ]:
small_data = load_coco_data(max_train=50)

small_rnn_model = CaptioningRNN(cell_type='rnn',
                                word_to_idx=data['word_to_idx'],
                                input_dim=data['train_features'].shape[1],
                                hidden_dim=512,
                                wordvec_dim=256,
                               )

small_rnn_solver = CaptioningSolver(small_rnn_model,small_data,
                                   update_rule='adam',
                                   num_epochs=50,
                                   batch_size=25,
                                   optim_config={'learning_rate':5e-3},
                                   lr_decay=0.95,
                                   verbose=True,print_every=10)

small_rnn_solver.train()
plt.plot(small_rnn_solver.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training loss history')
plt.show()

In [ ]:
for split in ['train','val']:
    minibatch = sample_coco_minibatch(small_data,split,batch_size=2)
    gt_captions, features, urls = minibatch
    gt_captions = decode_captions(gt_captions,data['idx_to_word'])
    
    sample_captions = small_rnn_model.sample(features)
    sample_captions = decode_captions(sample_captions,data['idx_to_word'])
    
    for gt_caption, sample_caption, url in zip(gt_captions,sample_captions,urls):
        plt.imshow(image_from_url(url))
        plt.title('%s\n%s\nGT:%s' %(split,sample_caption,gt_caption))
        plt.axis('off')
        plt.show()